# 경사 하강법을 이용한 얕은 신경망 학습


In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [2]:
EPOCHS = 100

## 네트워크 구조 정의
### 얕은 신경망
#### 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [4]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables) # df(x)/dx
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_metric(labels, predictions)

## 데이터셋 생성, 전처리

In [5]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10, 2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)
        
pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)


## 모델 생성

In [6]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프

In [9]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch: {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

Epoch: 1, Loss: 2.2901194095611572, Accuracy: 17.0
Epoch: 2, Loss: 2.0682685375213623, Accuracy: 30.25
Epoch: 3, Loss: 1.9094949960708618, Accuracy: 38.400001525878906
Epoch: 4, Loss: 1.7862486839294434, Accuracy: 44.92500305175781
Epoch: 5, Loss: 1.6874048709869385, Accuracy: 49.52000045776367
Epoch: 6, Loss: 1.6043163537979126, Accuracy: 53.60000228881836
Epoch: 7, Loss: 1.5341240167617798, Accuracy: 56.98571014404297
Epoch: 8, Loss: 1.4723236560821533, Accuracy: 59.5625
Epoch: 9, Loss: 1.4170167446136475, Accuracy: 62.12221908569336
Epoch: 10, Loss: 1.3668104410171509, Accuracy: 64.25
Epoch: 11, Loss: 1.3224502801895142, Accuracy: 66.0
Epoch: 12, Loss: 1.2820253372192383, Accuracy: 67.51667022705078
Epoch: 13, Loss: 1.2444525957107544, Accuracy: 68.75384521484375
Epoch: 14, Loss: 1.2099686861038208, Accuracy: 69.97142791748047
Epoch: 15, Loss: 1.1776034832000732, Accuracy: 71.06666564941406
Epoch: 16, Loss: 1.1477738618850708, Accuracy: 72.05000305175781
Epoch: 17, Loss: 1.119807243

## 데이터셋 및 학습 파라미터 저장

In [12]:
np.savez_compressed('ch2_dataset.npz', inputs=pts, labels=labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                    W_h=W_h,
                    b_h=b_h,
                    W_o=W_o,
                    b_o=b_o)